In [1]:
import torch
import pickle

with open( "x_train_hist.p", "rb" ) as file:
    x_train_hist = pickle.load(file)
    
with open( "y_train_hist.p", "rb" ) as file:
    y_train_hist = pickle.load(file)
    
with open( "x_valid_hist.p", "rb" ) as file:
    x_valid_hist = pickle.load(file)
    
with open( "y_valid_hist.p", "rb" ) as file:
    y_valid_hist = pickle.load(file)

# Load data

In [2]:
def load_data(indices):
    x = None
    
    for idx in tqdm(indices):
        t1 = dataset._get_sample_dict_from_subject(dataset.subjects[idx])["T1"].data
        margin = int(t1.shape[-1] * 0.1)
        t1 = t1[:,:,:,margin:-margin]

        t2 = dataset._get_sample_dict_from_subject(dataset.subjects[idx])["T2"].data
        margin = int(t2.shape[-1] * 0.1)
        t2 = t2[:,:,:,margin:-margin]

        data = torch.cat((t1, t2), -1)

        if x is None:
            x = data
        else:    
            x = torch.cat((x, data), 0)
            
    return x

x_train = load_data(i_train)
x_valid = load_data(i_valid)

NameError: name 'i_train' is not defined

In [ ]:
from mriqa_dataset import simulate_artefacts

# Augment

In [ ]:
import numpy as np
from collections import defaultdict

param_num_slices = 30

def augment(x):
    x_artefact = defaultdict(lambda: defaultdict(list))

    for index, patient in tqdm(enumerate(x), total=len(x), leave=False):
        for label in tqdm(range(1,5), leave=False):
            artefact = np.zeros(5)
            artefact[label] = 1
            
            slices = np.random.choice(120,param_num_slices,replace=False)

            x_artefact[index][label].append(simulate_artefacts(patient[:,:,slices].reshape((1,256,256,param_num_slices)), artefact[1:]))
            
    return x_artefact

In [ ]:
a_train = augment(x_train)

In [ ]:
a_valid = augment(x_valid)

In [ ]:
import torch
import random
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

from mriqa_dataset import MRIQADataset
from networks import ClassicCNN

net = ClassicCNN(num_classes=5)
net = net.cuda()

optimizer = optim.Adam(net.parameters())
ce = CrossEntropyLoss().cuda()

In [ ]:
random.seed(21062020)
np.random.seed(21062020)
torch.manual_seed(21062020)
torch.cuda.manual_seed(21062020)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(torch.cuda.current_device())
torch.cuda.set_device(0)

In [ ]:
from random import shuffle

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def make_batch(x_data, a_data):
    x_out = None
    y_out = None
    
    indices = np.arange(len(x_data))
    shuffle(indices)
    
    for i in indices:
        x = x_data[i]
        x_slice = np.random.randint(x.shape[-1])
        data = x[:,:,x_slice].reshape(1, 1, 256, 256)
        data -= torch.min(data)
        data /= torch.max(data)
        
        label = torch.tensor([0])
        
        if x_out is None:
            x_out = data
            y_out = label
            
        else:
            x_out = torch.cat((x_out, data), 0)
            y_out = torch.cat((y_out, label), 0)
            
    for index in indices:
        for artefact in range(1,5):
            
            x = a_data[index][artefact][0]
            
            x_slice = np.random.randint(x.shape[-1])
            data = x[0, :,:,x_slice].reshape(1, 1, 256, 256)
            
            data -= torch.min(data)
            data /= torch.max(data)

            label = torch.tensor([artefact])

            if x_out is None:
                x_out = data
                y_out = label

            else:
                x_out = torch.cat((x_out, data), 0)
                y_out = torch.cat((y_out, label), 0)
    
    x_out, y_out = unison_shuffled_copies(x_out, y_out)
    
    return x_out, y_out

In [ ]:
x_t, y_t = make_batch(x_train, a_train)
x_v, y_v = make_batch(x_valid, a_valid)

In [ ]:
batch_size = 16

max_epochs = 10

for epoch in range(max_epochs):
    net.train()
    epoch_loss = 0.0

    for i in range(0, x_t.shape[0], batch_size):
        sample = x_t[i:i+batch_size]
        sample = sample.cuda()

        label = y_t[i:i+batch_size]
        label = label.cuda()

        prediction = net(sample)
        loss = ce(prediction, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch}: Train Loss {epoch_loss}')

In [ ]:
epoch_loss = 0.0
net.train()

# train loop
for sample, label in tqdm(train_loader, total=len(train_loader)):
    sample = sample.cuda()
    label = label.cuda()

    prediction = net(sample)
    loss = ce(prediction, label)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

print('[{}] train-loss: {}'.format(epoch, epoch_loss / num_mini_batches))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(a_valid[80][4][0][0,:,:,5])
plt.show()

In [ ]:
a_valid[0][1][0][0,:,:,6] == a_valid[0][1][0][0,:,:,5]

In [ ]:
x_artefact

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.imshow(x_artefact[4][0,:,:,100])
plt.show()

In [ ]:
x_artefact[4][0,:,:,100]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.imshow(x_train[0,:,:,100])
plt.show()

In [ ]:
x_ -= torch.min(x_)
x_ /= torch.max(x_)

from numpy import random
margin = int(x_.shape[-1] * 0.1)
slice_number = random.randint(margin, x_.shape[-1] - margin)
x_ = x_[..., slice_number].unsqueeze(dim=0)
x_ = x_.squeeze(dim=0)
x_ = x_.squeeze(dim=0)

import matplotlib.pyplot as plt
plt.imshow(x_)

In [ ]:
x_.shape

In [ ]:
x_train.shape

In [ ]:
x_train[0]

In [ ]:
import numpy as np
x_train = torch.FloatTensor(np.array(x_train))

In [ ]:
x_train.data.shape

In [ ]:
x.shape

In [ ]:
subject = train_set[0]
sample = train_set._get_sample_dict_from_subject(subject)
x = sample['T1'].data